Declaración de librerías

In [ ]:
import os
#import kagglehub
import pandas as pd

In [ ]:
# Download latest version
#path = kagglehub.dataset_download("imdevskp/corona-virus-report")
#print("Path to dataset files:", path)
# Listar los archivos en el directorio
#files = os.listdir(path)
#print("Archivos en el directorio:", files)
#dataset_file = os.path.join(path, files[4])
#df = pd.read_csv(dataset_file)
#df.head()

Lectura de archivo

In [ ]:
#https://ourworldindata.org/covid-cases
df = pd.read_csv('owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

Contagios vs Muertes : Joel Ramírez

In [ ]:
# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el mes utilizando una estrategia diferente
conf_deaths_df.loc[:, 'month'] = conf_deaths_df['date_format'].dt.to_period('M')

# Agrupar por mes y sumar los valores de 'new_cases' y 'new_deaths'
grouped_month_df = conf_deaths_df.groupby('month').agg({'new_cases': 'sum', 'new_deaths': 'sum'})
# Mostrar las primeras filas del DataFrame agrupado por mes
grouped_month_df.head(5)


,new_cases,new_deaths
month,,
2020-01,8140.0,254.0
2020-02,304998.0,9643.0
2020-03,2775012.0,169349.0
2020-04,8618449.0,783274.0
2020-05,12841220.0,762141.0


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el trimestre
conf_deaths_df.loc[:, 'quarter'] = conf_deaths_df['date_format'].dt.to_period('Q')

# Agrupar por trimestre y sumar los valores de 'new_cases' y 'new_deaths'
grouped_quarter_df = conf_deaths_df.groupby('quarter').agg({'new_cases': 'sum', 'new_deaths': 'sum'}).reset_index()

# Función para ajustar los valores y las etiquetas
def adjust_values(values):
    return values / 1e6, ' (en millones)'

# Ajustar los valores de new_cases y new_deaths
grouped_quarter_df['new_cases_adj'], cases_label = adjust_values(grouped_quarter_df['new_cases'])
grouped_quarter_df['new_deaths_adj'], deaths_label = adjust_values(grouped_quarter_df['new_deaths'])

# Asegúrate de que 'quarter' sea un string
grouped_quarter_df['quarter'] = grouped_quarter_df['quarter'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_cases'
fig.add_trace(go.Scatter(
    x=grouped_quarter_df['quarter'],
    y=grouped_quarter_df['new_cases_adj'],
    name='Nuevos Casos',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='green'),
    text=grouped_quarter_df['new_cases'].apply(lambda x: f'{x:,.0f}'),
    textposition='top center',
    textfont=dict(size=14)
))

# Añadir gráfico de scatter para 'new_deaths'
fig.add_trace(go.Scatter(
    x=grouped_quarter_df['quarter'],
    y=grouped_quarter_df['new_deaths_adj'],
    name='Nuevas Muertes',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='red'),
    text=grouped_quarter_df['new_deaths'].apply(lambda x: f'{x:,.0f}'),
    textposition='bottom center',
    textfont=dict(size=14)
))

# Añadir etiquetas y título
fig.update_layout(
    title='Nuevos Casos y Muertes Trimestrales',
    xaxis=dict(
        title='Trimestre',
        title_standoff=30,
        tickangle=-45
    ),
    yaxis=dict(
        title=f'Conteo {cases_label}',  # Etiqueta del eje y
        tickformat=',.0f'  # Formato de número con comas
    ),
    legend_title='Métricas',
)

# Mostrar el gráfico
fig.show()

# Guardar el gráfico como un archivo HTML
pio.write_html(fig, file='nuevos_casos_y_muertes_trimestrales.html', auto_open=True)


# VACUNADOS POR SEMESTRE

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + '-S' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_vaccinations' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_vaccinations': 'sum'}).reset_index()

# Función para ajustar los valores y las etiquetas
def adjust_values(values):
    return values / 1000000, ' (en millones)'

# Ajustar los valores de new_vaccinations y new_deaths
grouped_semestre_df['new_vaccinations_adj'], cases_label = adjust_values(grouped_semestre_df['new_vaccinations'])

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_vaccinations'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_vaccinations_adj'],
    name='Vacunados',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='green'),
    text=grouped_semestre_df['new_vaccinations'].apply(lambda x: f'{x:,.0f}'),  # Añadir valores en las líneas con formato
    textposition='top center',
    textfont=dict(size=14)
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title='Vacunados por Semestre',
    xaxis=dict(
        title='Semestre',
        title_standoff=30,
        tickangle=-45
    ),
    yaxis=dict(
        title=f'Conteo {cases_label}',  # Etiqueta del eje y
        tickformat=',.0f'  # Formato de número con comas
    ),
    legend=dict(
        x=0.5,  # Posición horizontal de la leyenda
        y=1.1,  # Posición vertical de la leyenda
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=12,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='black',
        borderwidth=2
    ),
    paper_bgcolor='white'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()

# Guardar el gráfico como un archivo HTML
pio.write_html(fig, file='nuevos_casos_y_muertes_semestrales.html', auto_open=True)


# CASOS CONFIRMADOS

Plotly

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + '-S' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_cases' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_cases': 'sum'}).reset_index()

# Función para ajustar los valores y las etiquetas
def adjust_values(values):
    return values / 1000000, ' (en millones)'

# Ajustar los valores de new_cases y new_deaths
grouped_semestre_df['new_cases_adj'], cases_label = adjust_values(grouped_semestre_df['new_cases'])

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_cases'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_cases_adj'],
    name='Confirmados',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='blue'),
    text=grouped_semestre_df['new_cases'].apply(lambda x: f'{x:,.0f}'),  # Añadir valores en las líneas con formato
    textposition='top center',
    textfont=dict(size=14)
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title='Confirmados por Semestre',
    xaxis=dict(
        title='Semestre',
        title_standoff=30,
        tickangle=-45
    ),
    yaxis=dict(
        title=f'Conteo {cases_label}',  # Etiqueta del eje y
        tickformat=',.0f'  # Formato de número con comas
    ),
    legend=dict(
        x=0.5,  # Posición horizontal de la leyenda
        y=1.1,  # Posición vertical de la leyenda
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=12,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='black',
        borderwidth=2
    ),
    paper_bgcolor='white'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()

# Guardar el gráfico como un archivo HTML
pio.write_html(fig, file='nuevos_casos_y_muertes_semestrales.html', auto_open=True)


# CASOSO MUERTES

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Copia independiente del DataFrame original
conf_deaths_df = conf_deaths_df.copy()

# Convertir a datetime
conf_deaths_df.loc[:, 'date_format'] = pd.to_datetime(conf_deaths_df['date'])

# Crear una nueva columna para el semestre
conf_deaths_df.loc[:, 'semestre'] = (conf_deaths_df['date_format'].dt.year.astype(str) + '-S' + ((conf_deaths_df['date_format'].dt.month - 1) // 6 + 1).astype(str))

# Agrupar por semestre y sumar los valores de 'new_deaths' y 'new_deaths'
grouped_semestre_df = conf_deaths_df.groupby('semestre').agg({'new_deaths': 'sum'}).reset_index()

# Función para ajustar los valores y las etiquetas
def adjust_values(values):
    return values / 1000000, ' (en millones)'

# Ajustar los valores de new_deaths y new_deaths
grouped_semestre_df['new_deaths_adj'], cases_label = adjust_values(grouped_semestre_df['new_deaths'])

# Asegúrate de que 'semestre' sea un string
grouped_semestre_df['semestre'] = grouped_semestre_df['semestre'].astype(str)

# Crear la figura
fig = go.Figure()

# Añadir gráfico de scatter para 'new_deaths'
fig.add_trace(go.Scatter(
    x=grouped_semestre_df['semestre'],
    y=grouped_semestre_df['new_deaths_adj'],
    name='Muertes',
    mode='lines+markers+text',  # Mostrar los valores en las líneas
    marker=dict(color='red'),
    text=grouped_semestre_df['new_deaths'].apply(lambda x: f'{x:,.0f}'),  # Añadir valores en las líneas con formato
    textposition='top center',
    textfont=dict(size=14)
))

# Añadir etiquetas, título y configurar la leyenda
fig.update_layout(
    title='Muertes por Semestre',
    xaxis=dict(
        title='Semestre',
        title_standoff=30,
        tickangle=-45
    ),
    yaxis=dict(
        title=f'Conteo {cases_label}',  # Etiqueta del eje y
        tickformat=',.0f'  # Formato de número con comas
    ),
    legend=dict(
        x=0.5,  # Posición horizontal de la leyenda
        y=1.1,  # Posición vertical de la leyenda
        traceorder='normal',
        font=dict(
            family="Arial, sans-serif",
            size=12,
            color="black"
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='black',
        borderwidth=2
    ),
    paper_bgcolor='white'  # Fondo blanco para el papel
)

# Mostrar el gráfico
fig.show()

# Guardar el gráfico como un archivo HTML
pio.write_html(fig, file='nuevos_casos_y_muertes_semestrales.html', auto_open=True)


OLD Gráfico combinado plotly